# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import urllib.request
import zipfile
import os

#Fill this in based on where you saved the training and testing data

data_set_zip_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip"
file_name = './traffic-signs-data-new.zip'

if not os.path.isfile(file_name):
    print("Downloading dataset from the web...")
    # download the dataset from the Web
    urllib.request.urlretrieve(data_set_zip_url, file_name)
    print("Extracting the archive...")
    # extract pickle files from the zip archive
    zip_ref = zipfile.ZipFile(file_name, 'r') 
    zip_ref.extractall('./')
    zip_ref.close()

#pickle files
training_file = 'train.p'
validation_file = 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

#numpy arrays of training, validation and test sets
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
# Number of training examples
n_train = X_train.shape[0] # 34799 

# Number of validation examples
n_validation = X_valid.shape[0] # 4410

# Number of testing examples.
n_test = X_test.shape[0] # 12630

# The shape of an traffic sign image: all the images are pre-processed to be of an equal size
image_shape = X_train.shape[1:] # 32x32x3 -- color image of 32x32 px

# Unique classes/labels in the dataset.
n_classes = len(set(y_train)) # 43

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
import numpy as np
import random
#create a 1-D human interpretable array of tuples of image classes
text_labels = np.recfromcsv('./signnames.csv', delimiter=',')
_, text_labels = zip(*text_labels)
#decode byte strings from labels
text_labels = list(map(lambda x : x.decode("utf-8"), text_labels))

In [ ]:
### Data exploration visualization code goes here.
import matplotlib.pyplot as plt
import math
# Visualizations will be shown in the notebook.
%matplotlib inline

#print 1 random image of every class of traffic signs
NUM_COLUMNS = 5
NUM_ROWS = math.ceil(n_classes / NUM_COLUMNS)
fig = plt.figure(figsize=(20,25))
for label in range(n_classes):
    idx = random.choice(list(np.where(y_train == label)[0]))
    img = X_train[idx]
    ax = fig.add_subplot(NUM_ROWS,NUM_COLUMNS,label+1)
    ax.imshow(img, interpolation='none')
    ax.set_title(text_labels[label],  fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("img #{}".format(idx))
plt.tight_layout(w_pad=-0.5, h_pad=+1.0)
plt.suptitle('A Random image for each traffic sign class', y=1.03, fontsize=30)
plt.show()

In [ ]:
#randomly print # num_images of one random class
random_class = random.choice(range(n_classes))
num_images = 24
NUM_COLUMNS = 6
NUM_ROWS = math.ceil(num_images / NUM_COLUMNS)
images = np.random.choice(list(np.where(y_train == random_class)[0]), num_images, replace=False)
fig = plt.figure(figsize=(6,6))
for idx, el in enumerate(images):
    img = X_train[el]
    ax = fig.add_subplot(NUM_ROWS,NUM_COLUMNS,idx+1)
    ax.imshow(img, interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])
plt.suptitle("Random images of \"{}\" traffic sign".format(text_labels[random_class]), y=0.85)
fig.tight_layout(w_pad=0.2, h_pad=-11.0)
plt.show()

In [ ]:
#print histograms of image classes in the data set
fig = plt.figure(figsize=(10,14))
ax = fig.add_subplot(111)
(n_train, bins_train, _) = plt.hist(y_train, 3*len(set(y_train)), histtype='bar', 
                                    align='mid', alpha=0.3, orientation='horizontal',
                                    label='training')
(n_test, bins_test, _) = plt.hist(y_test, 3*len(set(y_test)), histtype='bar', 
                                  align='mid', color='green', orientation='horizontal', alpha=0.5,
                                  label='test')
(n_valid, bins_valid, _) = plt.hist(y_valid, 3*len(set(y_valid)), histtype='bar', 
                                    align='mid',color='red', orientation='horizontal', alpha=0.7,
                                   label='validation')
plt.legend(bbox_to_anchor=(1.0,0.85))
ax.set_yticks(list(range(n_classes)))
ax.set_ylim([-1, n_classes])
ax.set_yticklabels(text_labels)
ax.set_xticks(list(range(0,2600,200)))
plt.gca().invert_yaxis()
ax.xaxis.tick_top()
plt.show()

Pre-processing of the data set. As some image classes are less represented then others (seen from the histogram above), we use algorithms for image processing from `skimage.exposure` and `skimage.transform` to augment image classes with filtered images. Namely, methods for histogram equalization, non-linear transforms and small rotation (7 degrees). 

In [ ]:
#filter functions for data augmentation
from functools import partial

def filter_eqhist(img):
    return skimage.exposure.equalize_hist(img)

def filter_eqadapthist(img):
    return skimage.exposure.equalize_adapthist(img, 17, 0.4)

def filter_adjustlog(img):
    return skimage.exposure.adjust_log(filter_eqadapthist(img), 0.8)

def filter_adjustsigmoid(img):
    return skimage.exposure.adjust_sigmoid(filter_eqadapthist(img), cutoff=0.4, gain=8)

def filter_rotate(img, func, angle):
    return skimage.transform.rotate(func(img), angle)
 
#rotated images of the filter functions
filter_eqadapthistright = partial(filter_rotate, func=filter_eqadapthist, angle=+7)
filter_eqadapthistleft = partial(filter_rotate, func=filter_eqadapthist, angle=-7)
filter_eqhistright = partial(filter_rotate, func=filter_eqadapthist, angle=+7)
filter_eqhistleft = partial(filter_rotate, func=filter_eqadapthist, angle=-7)
filter_adjustlogright = partial(filter_rotate, func=filter_adjustlog, angle=+7)
filter_adjustlogleft = partial(filter_rotate, func=filter_adjustlog, angle=-7)
filter_adjustsigmoidright = partial(filter_rotate, func=filter_adjustsigmoid, angle=+7)
filter_adjustsigmoidleft = partial(filter_rotate, func=filter_adjustsigmoid, angle=-7)

In [ ]:
# Showcase of filter functions
# Apply filters on a random image
import skimage.exposure 
import skimage.transform
fig = plt.figure(figsize=(16,3))

NUM_IMG = 13
img_idx = 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_raw = random.choice(X_train)
plt.imshow(pic_raw)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Original', fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_eqadapthist = filter_eqadapthist(pic_raw)
plt.imshow(pic_eqadapthist)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Filter {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_eqhist = filter_eqhist(pic_raw)
plt.imshow(pic_eqhist)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Filter {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_log = filter_adjustlog(pic_raw)
plt.imshow(pic_log)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Filter {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_sigmoid = filter_adjustsigmoid(pic_raw)
plt.imshow(pic_sigmoid)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Filter {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_eqadapthistright(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_eqadapthistleft(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_eqhistright(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_eqhistleft(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_adjustlogright(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_adjustlogleft(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_adjustsigmoidright(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)

img_idx += 1
ax = fig.add_subplot(1,NUM_IMG,img_idx)
pic_rotated = filter_adjustsigmoidleft(pic_raw)
plt.imshow(pic_rotated)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Rotate {}".format(img_idx-1), fontsize=9)
plt.show()

Exploring image underrepresented image classes

In [ ]:
#traffic sign classes in ascending order of image examples 
from collections import OrderedDict
n_train_ = list(filter(lambda x : x > 0, n_train))
n_train_dict = {idx: el for idx,el in enumerate(n_train_)}
X_train_summary = OrderedDict(sorted(n_train_dict.items(), key=lambda t: t[1]))
print(X_train_summary)

A function that returns an image filter

In [ ]:
def create_filter(filter_num):
    filters = [filter_eqhist,
               filter_eqadapthist,
               filter_adjustlog,
               filter_adjustsigmoid,
               filter_eqadapthistright,
               filter_eqadapthistleft,
               filter_eqhistright,
               filter_eqhistleft,
               filter_adjustlogright,
               filter_adjustlogleft,
               filter_adjustsigmoidright,
               filter_adjustsigmoidleft]
    return filters[filter_num]

In [ ]:
from tqdm import tqdm

def find_ids_of_img_class(dataset, img_class):
    return list(np.where(dataset == img_class)[0])

threshold = 1700
X_augmented = []
Y_augmented = []
for class_id, num_examples in X_train_summary.items():
    if num_examples < threshold:
        print("Augmenting dataset for class '{}', of '{}' elements".format(class_id, num_examples))
        img_ids = find_ids_of_img_class(y_train, class_id)
        num_filter = 0
        X_augmented_partial = []
        y_augmented_partial = []
        examples_total = num_examples
        while examples_total < threshold :
            img_filter = create_filter(num_filter)
            print("Applying '{}' for {} images of class {}".format(str(img_filter), str(examples_total), class_id))
            X_augmented_partial = list(map(lambda x: img_filter(x), tqdm(X_train[img_ids])))
            Y_augmented_partial = [class_id] * len(X_augmented_partial)
            examples_total += len(X_augmented_partial)
            X_augmented += X_augmented_partial
            Y_augmented += Y_augmented_partial
            num_filter += 1
        print("Augmented training set: {} images, {} labels".format(len(X_augmented),len(Y_augmented)))

X_augmented_np = np.array(X_augmented)
Y_augmented_np = np.array(Y_augmented)


In [ ]:
#concatenate augmented and original dataset
X_train_new = np.concatenate((X_train, X_augmented_np), axis=0)
Y_train_new = np.concatenate((y_train, Y_augmented_np), axis=0)

In [ ]:
#print histograms of image classes in the data set
fig = plt.figure(figsize=(10,14))
ax = fig.add_subplot(111)
(n_train, bins_train, _) = plt.hist(Y_train_new, 3*len(set(Y_train_new)), histtype='bar', 
                                    align='mid', alpha=0.3, orientation='horizontal',
                                    label='training')
(n_test, bins_test, _) = plt.hist(y_test, 3*len(set(y_test)), histtype='bar', 
                                  align='mid', color='green', orientation='horizontal', alpha=0.5,
                                  label='test')
(n_valid, bins_valid, _) = plt.hist(y_valid, 3*len(set(y_valid)), histtype='bar', 
                                    align='mid',color='red', orientation='horizontal', alpha=0.7,
                                   label='validation')
plt.legend(bbox_to_anchor=(1.0,0.85))
ax.set_yticks(list(range(n_classes)))
ax.set_ylim([-1, n_classes])
ax.set_yticklabels(text_labels)
ax.set_xticks(list(range(0,2600,200)))
plt.gca().invert_yaxis()
ax.xaxis.tick_top()
plt.show()

In [ ]:
myfilter = create_filter(8)
print(myfilter)
print(filter_eqhist)
fig = plt.figure(figsize=(3,3))

ax = fig.add_subplot(1,1,1)
pic_raw = random.choice(X_augmented_np)
plt.imshow(myfilter(pic_raw))
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
#save augmented training set
#pickle files
# new_training_file = 'train_new.p'

# with open(new_training_file, mode='wb') as f:
#     pickle.dump({"X_train_new" : X_train_new, "Y_train_new" : Y_train_new}, f, pickle.HIGHEST_PROTOCOL)


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
from sklearn.utils import shuffle

X_train_shffled, y_train_shuffled = shuffle(X_train_new, Y_train_new)

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
def normalize(X):
    return (X - 128) / 128.

### Model Architecture

In [ ]:
import tensorflow as tf

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
from tensorflow.contrib.layers import flatten

def LeNet_like_gtsrb(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1    
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv_l1_in_depth = 3
    conv_l1_out_depth = 6
    conv_l1_padding = 'VALID'
    conv_l1_filter_size = 5
    conv_l1_W_size = [conv_l1_filter_size,
                      conv_l1_filter_size,
                      conv_l1_in_depth,
                      conv_l1_out_depth]
    conv_l1_B_size = [conv_l1_out_depth]
    conv_l1_W = tf.Variable(tf.truncated_normal(conv_l1_W_size), mu=mu, sigma=sigma)
    conv_l1_B = tf.Variable(tf.zeros(conv_l1_B_size))
    conv_l1_stride = [1,1,1,1] 
    conv_l1 = tf.nn.conv2d(x, conv_l1_W, conv_l1_stride, conv_l1_padding) + conv_l1_B
    # TODO: Activation.
    conv_l1_relu = tf.nn.relu(conv_l1)
    
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv_l1_pooling_kernel = [1,2,2,1]
    conv_l1_pooling_stride = [1,2,2,1]
    conv_l1_pooling_padding = 'VALID'
    conv_l1_pooling = tf.nn.max_pool(conv_l1_relu, 
                                     conv_l1_pooling_kernel, 
                                     conv_l1_pooling_stride,
                                     conv_l1_pooling_padding)

    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv_l2_in_depth = 6
    conv_l2_out_depth = 16
    conv_l2_padding = 'VALID'
    conv_l2_filter_size = 5
    conv_l2_W_size = [conv_l2_filter_size,
                      conv_l2_filter_size,
                      conv_l2_in_depth,
                      conv_l2_out_depth]
    conv_l2_B_size = [conv_l2_out_depth]
    conv_l2_W = tf.Variable(tf.truncated_normal(conv_l2_W_size))
    conv_l2_B = tf.Variable(tf.zeros(conv_l2_B_size))
    conv_l2_stride = [1,1,1,1] 
    conv_l2 = tf.nn.conv2d(conv_l1_pooling, conv_l2_W, conv_l2_stride, conv_l2_padding) + conv_l2_B
    
    # TODO: Activation.
    conv_l2_relu = tf.nn.relu(conv_l2)
    
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv_l2_pooling_kernel = [1,2,2,1]
    conv_l2_pooling_stride = [1,2,2,1]
    conv_l2_pooling_padding = 'VALID'
    conv_l2_pooling = tf.nn.max_pool(conv_l2_relu, 
                                     conv_l2_pooling_kernel, 
                                     conv_l2_pooling_stride,
                                     conv_l2_pooling_padding)
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc_flat = flatten(conv_l2_pooling)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W_in_dim = 400
    fc1_W_out_dim = 120
    fc1_W = tf.Variable(tf.truncated_normal([fc1_W_in_dim, fc1_W_out_dim]))
    fc1_B = tf.Variable(tf.truncated_normal([fc1_W_out_dim]))
    fc1_nn = tf.matmul(fc_flat, fc1_W) + fc1_B
    # TODO: Activation.
    fc1_relu = tf.nn.relu(fc1_nn)
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W_in_dim = 120
    fc2_W_out_dim = 84
    fc2_W = tf.Variable(tf.truncated_normal([fc2_W_in_dim, fc2_W_out_dim]))
    fc2_B = tf.Variable(tf.truncated_normal([fc2_W_out_dim]))
    fc2_nn = tf.matmul(fc1_relu, fc2_W) + fc2_B
    # TODO: Activation.
    fc2_relu = tf.nn.relu(fc2_nn)
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W_in_dim = 84
    fc3_W_out_dim = 43
    fc3_W = tf.Variable(tf.truncated_normal([fc3_W_in_dim, fc3_W_out_dim]))
    fc3_B = tf.Variable(tf.truncated_normal([fc3_W_out_dim]))
    logits = tf.matmul(fc2_relu, fc3_W) + fc3_B
    return logits

#### Features and Labels
x is a placeholder for a batch of input images. 
y is a placeholder for a batch of output labels.

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
rate = 0.001

logits = LeNet_like_gtsrb(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")